# Eat Safe, Love

## Notebook Set Up

In [14]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd


In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our database
print(db.list_collection_names())


['establishments']


In [18]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [19]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
results = establishments.count_documents(query)
print("The number of establishments who received a hygiene score of 20 is: ", results)


# Display the first document in the results using pprint
print("-"*20)
print("First Result:")
print("-"*20)
pprint(establishments.find_one(query))


The number of establishments who received a hygiene score of 20 is:  0
--------------------
First Result:
--------------------
None


In [20]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(query))
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(hygiene_df))
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

Rows in DataFrame:  0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [21]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': "London"}, 'RatingValue': {"$gte": '4'}}

# Use count_documents to display the number of documents in the result
results = establishments.count_documents(query)
print("The number of establishments in London with a rating greater than or equal to 4 is: ", results)
# Display the first document in the results using pprint
print("-"*20)
print("First Result:")
print("-"*20)
pprint(establishments.find_one(query))

The number of establishments in London with a rating greater than or equal to 4 is:  0
--------------------
First Result:
--------------------
None


In [22]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(establishments.find(query))
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(london_df))
# Display the first 10 rows of the DataFrame
london_df.head(10)

Rows in DataFrame:  0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [23]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.086058
longitude = 1.196408

query = {'RatingValue': '5', 'geocode.longitude': {'$gte': longitude - degree_search, 
         '$lte': longitude + degree_search}, 
         'geocode.latitude': {'$gte': latitude - degree_search,
        '$lte': latitude + degree_search}}
sort =  [('scores.Hygiene', 1)]
limit = 5
results = list(establishments.find(query).sort(sort).limit(limit))
# Print the results
pprint(results)


[]


In [24]:
# Convert result to Pandas DataFrame
top_5_df = pd.DataFrame(results)
top_5_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [25]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene':0}}

# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': -1}}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
print("total documents in the results:", len(results))
# Print the first 10 results
pprint(results[0:10])


total documents in the results: 0
[]


In [26]:
# Convert the result to a Pandas DataFrame
poor_hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(poor_hygiene_df))

# Display the first 10 rows of the DataFrame
poor_hygiene_df.head(10)


Rows in DataFrame:  0


""
